In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [16]:
spark = SparkSession.builder\
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.jars", "/Drivers/SQL_Sever/jdbc/sqljdbc42.jar")\
    .enableHiveSupport()\
    .getOrCreate()

In [20]:
spark.sql("create namespace gold;")

DataFrame[]

In [21]:
spark.sql("show namespaces;").show()

+---------+
|namespace|
+---------+
|   bronze|
|  default|
|     gold|
|    sales|
|   silver|
+---------+



In [18]:
spark.sql("use silver")

DataFrame[]

In [19]:
spark.sql("select count(distinct CustomerKey) from factsales").show()

+---------------------------+
|count(DISTINCT CustomerKey)|
+---------------------------+
|                      19119|
+---------------------------+



In [40]:
spark.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|   silver|dimcustomer|      false|
|   silver|    dimdate|      false|
|   silver|dimemployee|      false|
|   silver| dimproduct|      false|
|   silver|  factsales|      false|
+---------+-----------+-----------+



In [41]:
spark.sql("describe FactSales").show()

+-------------------+-------------+-------+
|           col_name|    data_type|comment|
+-------------------+-------------+-------+
|       SalesOrderID|          int|   NULL|
|        CustomerKey|          int|   NULL|
|        EmployeeKey|          int|   NULL|
|         ProductKey|          int|   NULL|
|     RevisionNumber|          int|   NULL|
|           OrderQty|     smallint|   NULL|
|          UnitPrice|decimal(19,4)|   NULL|
|  UnitPriceDiscount|decimal(19,4)|   NULL|
|           SubTotal|decimal(19,4)|   NULL|
|             TaxAmt|decimal(19,4)|   NULL|
|            Freight|decimal(19,4)|   NULL|
|           TotalDue|decimal(19,4)|   NULL|
|          OrderDate|    timestamp|   NULL|
|            DueDate|    timestamp|   NULL|
|           ShipDate|    timestamp|   NULL|
|             Status|          int|   NULL|
|    OnlineOrderFlag|      boolean|   NULL|
|   SalesOrderNumber|       string|   NULL|
|PurchaseOrderNumber|       string|   NULL|
|      AccountNumber|       stri

In [22]:
profit =spark.sql("""
  SELECT
    fs.*,
    (fs.SubTotal - fs.TaxAmt - fs.Freight) AS Profit
  FROM
    FactSales fs
""")
profit.select("SalesOrderID","CustomerKey", "EmployeeKey", "ProductKey","profit").show(5)
profit.write.format("hive").mode("overwrite").saveAsTable("gold.profit")


+------------+-----------+-----------+----------+----------+
|SalesOrderID|CustomerKey|EmployeeKey|ProductKey|    profit|
+------------+-----------+-----------+----------+----------+
|       43662|      29994|        282|       762|25190.1254|
|       43662|      29994|        282|       765|25190.1254|
|       43662|      29994|        282|       756|25190.1254|
|       43665|      29580|        283|       778|12546.8465|
|       43666|      30052|        276|       756| 4418.1228|
+------------+-----------+-----------+----------+----------+
only showing top 5 rows



In [28]:
Customer_Lifetime_Value=spark.sql("""
  SELECT
    fs.CustomerKey,
    SUM(fs.TotalDue) AS TotalRevenue,
    COUNT(DISTINCT fs.SalesOrderID) AS NumberOfOrders,
    cast((SUM(fs.TotalDue) / COUNT(DISTINCT fs.SalesOrderID)) as decimal(10,4)) as CLV
  FROM
    FactSales fs
  GROUP BY
    fs.CustomerKey
""")
Customer_Lifetime_Value.orderBy("CLV", ascending=False).show(truncate=False)
Customer_Lifetime_Value.write.format("hive").mode("overwrite").saveAsTable("gold.Customer_Lifetime_Value")

+-----------+-------------+--------------+-----------+
|CustomerKey|TotalRevenue |NumberOfOrders|CLV        |
+-----------+-------------+--------------+-----------+
|29509      |11990085.2908|12            |999173.7742|
|29724      |11844575.4884|12            |987047.9574|
|29543      |7894329.8593 |8             |986791.2324|
|29922      |3916111.3781 |4             |979027.8445|
|30059      |3819502.4623 |4             |954875.6156|
|30051      |3817925.4284 |4             |954481.3571|
|30113      |7626772.2207 |8             |953346.5276|
|29524      |3765284.0561 |4             |941321.0140|
|29841      |3719403.4521 |4             |929850.8630|
|29710      |3711002.0662 |4             |927750.5166|
|29510      |3682968.0277 |4             |920742.0069|
|29595      |3674735.5700 |4             |918683.8925|
|29935      |7308058.8694 |8             |913507.3587|
|29534      |3641722.8158 |4             |910430.7040|
|29942      |3578625.0448 |4             |894656.2612|
|29561    

In [29]:
Emp_Sales_performace =spark.sql("""
  SELECT
    e.EmployeeID,
    e.EmployeeName,
    SUM(fs.TotalDue) AS TotalSales,
    cast(AVG(fs.TotalDue) AS DECIMAL(10,4)) as AvgOrderValue,
    COUNT(DISTINCT fs.SalesOrderID) AS TotalOrders 
    FROM FactSales fs
    
    inner join dimemployee e
    ON fs.EmployeeKey = e.EmployeeID
    
    GROUP BY e.EmployeeID, e.EmployeeName
""")
Emp_Sales_performace.orderBy("TotalOrders",ascending =False).show()
Emp_Sales_performace.write.format("hive").mode("overwrite").saveAsTable("gold.Emp_Sales_performace")

+----------+--------------------+--------------+-------------+-----------+
|EmployeeID|        EmployeeName|    TotalSales|AvgOrderValue|TotalOrders|
+----------+--------------------+--------------+-------------+-----------+
|       277|      Jillian Carson|347868654.9495|   44456.0581|        473|
|       275|      Michael Blythe|327853784.2760|   46379.0896|        450|
|       279|         Tsvi Reiter|201612866.2556|   37218.5465|        429|
|       276|      Linda Mitchell|381125560.6115|   53626.7849|        418|
|       289|             Jae Pak|366037331.2842|   54324.3294|        348|
|       282|        José Saraiva|206045854.0384|   46438.1010|        271|
|       281|             Shu Ito|225255391.3744|   49561.1422|        242|
|       278|      Garrett Vargas|102841549.2822|   31315.9407|        234|
|       283|      David Campbell|123090545.2713|   54779.9489|        189|
|       290|Ranjit Varkey Chu...|190218560.7085|   55635.7300|        175|
|       284|    Tete Mens

In [30]:
product_trends =spark.sql("""
  SELECT
    p.ProductID,
    p.productName,
    d.Year,
    d.Month,
    SUM(fs.TotalDue) AS MonthlySales
  FROM FactSales fs

  inner join dimproduct p
  ON fs.ProductKey = p.ProductID
  
  inner join dimdate d
  ON fs.OrderDate = d.DateKey
  
  GROUP BY
    p.ProductID, p.productName, d.Year, d.Month
    
  ORDER BY
    p.ProductID, d.Year, d.Month
""")

product_trends.show()
product_trends.write.format("hive").mode("overwrite").saveAsTable("gold.product_trends")

+---------+--------------------+----+-----+------------+
|ProductID|         productName|Year|Month|MonthlySales|
+---------+--------------------+----+-----+------------+
|      707|Sport-100 Helmet,...|2011|   05| 246964.6144|
|      707|Sport-100 Helmet,...|2011|   07| 781782.7504|
|      707|Sport-100 Helmet,...|2011|   08|1306422.5044|
|      707|Sport-100 Helmet,...|2011|   10|2451107.6278|
|      707|Sport-100 Helmet,...|2011|   12| 123493.8543|
|      707|Sport-100 Helmet,...|2012|   01|1001293.6244|
|      707|Sport-100 Helmet,...|2012|   02| 489774.2332|
|      707|Sport-100 Helmet,...|2012|   03|1429122.2143|
|      707|Sport-100 Helmet,...|2012|   04| 679976.1760|
|      707|Sport-100 Helmet,...|2012|   05|1679032.2619|
|      707|Sport-100 Helmet,...|2012|   06|2520181.7941|
|      707|Sport-100 Helmet,...|2012|   07|2031765.9740|
|      707|Sport-100 Helmet,...|2012|   08|1189553.9608|
|      707|Sport-100 Helmet,...|2012|   09|1983843.2223|
|      707|Sport-100 Helmet,...

In [27]:
spark.sql("select * from dimdate").show(3)

+-------------------+----------+----+-----+---+-------+
|            DateKey|      Date|Year|Month|Day|Quarter|
+-------------------+----------+----+-----+---+-------+
|2011-10-10 00:00:00|2011-10-10|2011|   10| 10|      4|
|2011-12-04 00:00:00|2011-12-04|2011|   12| 04|      4|
|2012-01-22 00:00:00|2012-01-22|2012|   01| 22|      1|
+-------------------+----------+----+-----+---+-------+
only showing top 3 rows



In [28]:
spark.sql("show tables;").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|   silver|dimcustomer|      false|
|   silver|    dimdate|      false|
|   silver|dimemployee|      false|
|   silver| dimproduct|      false|
|   silver|  factsales|      false|
+---------+-----------+-----------+



In [31]:
customer_churn90 =spark.sql("""
    WITH ReferenceDate AS (
  SELECT DATE_SUB(MAX(OrderDate), 90) AS reference_date
  FROM FactSales
  ),

-- Retrieve customer keys who made purchases after the reference date
    purches_after_90 as(
    SELECT DISTINCT CustomerKey
    FROM FactSales
    WHERE OrderDate > (SELECT reference_date FROM ReferenceDate)
    ),
    
    customer_IDs_not_purches_last90 as (
    select CustomerKey from factsales
    where factsales.CustomerKey not in (select CustomerKey from  purches_after_90)
    ),
    
    fact as (
    select * from factsales c
    where c.customerkey in (select CustomerKey from customer_IDs_not_purches_last90)
    )
    
    SELECT
    dimcustomer.CustomerID,
    dimcustomer.CustomerName,
    MAX(fact.OrderDate) AS LastPurchaseDate
    from fact inner join dimcustomer
    on fact.CustomerKey = dimcustomer.customerid
    group by dimcustomer.CustomerID, dimcustomer.CustomerName
    
""")
customer_churn90.write.format("hive").mode("overwrite").saveAsTable("gold.customer_churn90")

In [32]:
spark.sql("use gold;")

DataFrame[]

In [35]:
spark.sql("show tables;").show(truncate=False)

+---------+-----------------------+-----------+
|namespace|tableName              |isTemporary|
+---------+-----------------------+-----------+
|gold     |customer_churn90       |false      |
|gold     |customer_lifetime_value|false      |
|gold     |emp_sales_performace   |false      |
|gold     |product_trends         |false      |
|gold     |profit                 |false      |
+---------+-----------------------+-----------+



In [36]:
spark.sql("use silver;")

DataFrame[]